### Import data

In [1]:
from dataprocessing import OBR, PT1000, Results
import numpy as np
import pandas as pd

range_bor = [0.8, 1]
range_ger = [2.04 , 2.2]

temprange_obr_output = 'M:/Intern/OE300/E2_OE320/85_Hiwi_Ordner/gmn-js/20240703_Bachelorarbeit/Results/final validation/joined/CSV'
temprange_txt = 'M:/Intern/OE300/E2_OE320/85_Hiwi_Ordner/gmn-js/20240703_Bachelorarbeit/Results/final validation/timestamps_test.txt'
temprange_pt1000_outputs = 'M:/Intern/OE300/E2_OE320/85_Hiwi_Ordner/gmn-js/20240703_Bachelorarbeit/Results/final validation//merged_without_nan.csv'


### Bor fiber

In [2]:
obr_result_bor = OBR(temprange_obr_output, "Save", range=range_bor)
obr_result_bor.plot_animated_chart("obr result bor")

Lower limit: -0.031230374999999998, Upper Limit: 0.021728624999999998
0 outliers removed
Lower limit: -97.51527662500001, Upper Limit: -22.88165962499999
0 outliers removed
Lower limit: -97.33738100000001, Upper Limit: -23.068395
0 outliers removed
Lower limit: -97.59492437500002, Upper Limit: -22.98806937499999
0 outliers removed
Lower limit: -97.953259625, Upper Limit: -22.771824625000004
0 outliers removed
Lower limit: -98.61940387500002, Upper Limit: -22.002180874999986
0 outliers removed
Lower limit: -0.02651825, Upper Limit: 0.021767750000000002
1 outliers removed
Lower limit: -0.018301375, Upper Limit: 0.024687624999999998
1 outliers removed
Lower limit: -0.025115000000000002, Upper Limit: 0.020053
0 outliers removed
Lower limit: -59.788452625000005, Upper Limit: -10.938597624999996
0 outliers removed
Lower limit: -58.852073874999995, Upper Limit: -12.221148875000004
0 outliers removed
Lower limit: -58.9890455, Upper Limit: -12.127749499999997
0 outliers removed
Lower limit: -59

### Ger fiber

In [4]:
obr_result_ger = OBR(temprange_obr_output, "Save",range=range_ger)
obr_result_ger.plot_animated_chart("obr result ger")

Lower limit: -0.02725725, Upper Limit: 0.014484750000000001
0 outliers removed
Lower limit: -135.71996825, Upper Limit: -8.110304249999992
3 outliers removed
Lower limit: -134.90095437500003, Upper Limit: -9.458507374999996
3 outliers removed
Lower limit: -136.65977025, Upper Limit: -8.391690250000003
3 outliers removed
Lower limit: -132.37793937499998, Upper Limit: -12.247598375000003
2 outliers removed
Lower limit: -134.11848, Upper Limit: -11.259286000000003
3 outliers removed
Lower limit: -0.026479999999999997, Upper Limit: 0.01215
0 outliers removed
Lower limit: -0.022662750000000002, Upper Limit: 0.02468725
0 outliers removed
Lower limit: -0.0137065, Upper Limit: 0.0062295
4 outliers removed
Lower limit: -68.021861625, Upper Limit: 2.4084713750000013
0 outliers removed
Lower limit: -66.687396, Upper Limit: 1.2825699999999998
0 outliers removed
Lower limit: -67.352878, Upper Limit: 1.8145660000000028
0 outliers removed
Lower limit: -67.52999537500001, Upper Limit: 2.08120362500001

In [6]:
temprange_pt1000 = PT1000(temprange_pt1000_outputs)
temprange_pt1000.plot("test")

            timestamps  Temperature        Strain
0  2024-11-25 12:18:30       295.45 -6.847000e-07
1  2024-11-25 12:18:31       295.42 -6.847000e-07
2  2024-11-25 12:18:32       295.45 -6.847000e-07
3  2024-11-25 12:18:33       295.42 -5.520000e-07
4  2024-11-25 12:18:34       295.45 -9.290000e-07


### Results and validation

In [7]:
bor_results = Results(obr_result_bor, temprange_pt1000,temprange_txt, "Save")

m:\Intern\OE300\E2_OE320\85_Hiwi_Ordner\gmn-js\20240703_Bachelorarbeit\Scripts\Analysis\dataprocessing.py:182: FutureWarning:

DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.



In [8]:
ger_results = Results(obr_result_ger, temprange_pt1000,temprange_txt, "Save")

m:\Intern\OE300\E2_OE320\85_Hiwi_Ordner\gmn-js\20240703_Bachelorarbeit\Scripts\Analysis\dataprocessing.py:182: FutureWarning:

DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.



Since it was acquired 5 samples for each sample in the validation experiment, the code block below performs the averaging of the temperature, strain and spectral shift of the 5 samples.

In [70]:

timestamps = ger_results.df["timestamps"].iloc[::5].reset_index(drop=True)
timestep = ger_results.df["timestep"].iloc[::5].reset_index(drop=True)
numeric_columns = ger_results.df.drop(columns=["timestamps", "timestep"]).select_dtypes(include=[np.number])

averaged_numeric = numeric_columns.groupby(np.arange(len(numeric_columns)) // 5).mean().reset_index(drop=True)

df_validation_ger = pd.concat([timestamps, timestep, averaged_numeric], axis=1)

timestamps = bor_results.df["timestamps"].iloc[::5].reset_index(drop=True)
timestep = bor_results.df["timestep"].iloc[::5].reset_index(drop=True)
numeric_columns = bor_results.df.drop(columns=["timestamps", "timestep"]).select_dtypes(include=[np.number])

averaged_numeric = numeric_columns.groupby(np.arange(len(numeric_columns)) // 5).mean().reset_index(drop=True)

df_validation_bor = pd.concat([timestamps, timestep, averaged_numeric], axis=1)


In [ ]:
df_validation_ger.head()

In [ ]:
df_validation_bor.head()

In [91]:
#functions created to calculate temperature with and without decoupling
from decoupling import solve_delta, solve_delta_without_decoupling  
#Constants needed to convert spectral shift from GHz to wavelengh 
c = 299792458 # Speed of light
center_wl = 1306 # Central wavelenght of Luna OBR

T_ref = 22 + 273
delta_lambda_bor = df_validation_bor["dv"][1]
delta_lambda_ger = df_validation_ger["dv"][1]


delta_lambda = np.array([[(delta_lambda_ger* center_wl) / c], [(delta_lambda_bor* center_wl) / c]])
Ke1, Ke2, Kt1, Kt2 = 8.29*10**-1, 8.29*10**-1, 5.35*10**-6, 5.15*10**-6  # Constants taken from K_T estimation results


The code block below uses this function to apply equation 4 (pg. 23) of the thesis

In [94]:
delta_epsilon, delta_T = solve_delta(delta_lambda, Ke1, Ke2, Kt1, Kt2)

Now that we have the strain difference, we add it to the initial strain:

In [75]:
strain_ref = df_validation_bor["Strain"][0]
calculated_strain = strain_ref + delta_epsilon
calculated_strain

array([0.00012893])

By subtracting the calculated strain from the actual measured strain, the absolute error is obtained (as displayed in table 4)

In [76]:
df_validation_bor["Strain"][1] - calculated_strain

array([-2.77285961e-05])

It was also calculated the temperature difference, so it is also added to the initial temperature:

In [95]:
calculated_T = T_ref + delta_T
calculated_T 

array([308.26559352])

Just like for strain, by subtracting the calculated temperature from the actual measured temperature, the absolute error is obtained (as displayed in table 4)

In [78]:
df_validation_bor["Temperature"][1] - calculated_T

array([-2.15359352])

Now, for comparison reasons, the following code blocks calculate the temperature variation applying equation 2, disconsidering any decoupling nor the thermo induced strain

In [103]:
delta_t_ger = solve_delta_without_decoupling(delta_lambda[0], Kt1)
delta_t_bor = solve_delta_without_decoupling(delta_lambda[1], Kt2)

In [104]:
#Temperature variation: delta_t
delta_t_ger, delta_t_bor

(array([33.39030395]), array([34.68701478]))

In [105]:
#Calculated temperature germanium fiber
calculated_T_ger = T_ref + delta_t_ger
calculated_T_ger 

array([328.39030395])

In [106]:
#Calculated temperature boron fiber
calculated_T_bor = T_ref + delta_t_bor
calculated_T_bor 

array([329.68701478])